In [1]:
# is base model we have here good enough? - we ended up using albert
# explore improvements
  # Maybe different flavor of bert? [X]
  # Maybe try a few different models and measure F1 scores?
  # better information retrieval? 
  # Maybe try and fine tune on a different dataset?
  # chunk in a way that we tend to get full sentances? Right now a chunk can stop at a random spot in the sentance
    
  # a few ideas that can help with first three improvemetns are here https://towardsdatascience.com/which-flavor-of-bert-should-you-use-for-your-qa-task-6d6a0897fb24

In [2]:
# script to fine tune the model (it's fine tuned on squad 2.0)

# !python run_squad.py  \
#     --model_type bert   \
#     --model_name_or_path bert-base-uncased  \
#     --output_dir models/bert/ \
#     --data_dir dataset/   \
#     --overwrite_output_dir \
#     --overwrite_cache \
#     --do_train  \
#     --train_file train-v2.0.json   \
#     --version_2_with_negative \
#     --do_lower_case  \
#     --do_eval   \
#     --predict_file dev-v2.0.json   \
#     --per_gpu_train_batch_size 2   \
#     --learning_rate 3e-5   \
#     --num_train_epochs 2.0   \
#     --max_seq_length 384   \
#     --doc_stride 128   \
#     --threads 10   \
#     --save_steps 5000 

In [3]:
import torch
import wikipedia
import tensorflow as tf
import numpy as np

from collections import OrderedDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TFAutoModelForQuestionAnswering

In [4]:
tokenizer = AutoTokenizer.from_pretrained("models/bert")
model = TFAutoModelForQuestionAnswering.from_pretrained("models/bert", from_pt=True)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the TF 2.0 model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [38]:
question = "When did world war 2 start?"

q_search = wikipedia.search(question)
page = wikipedia.page(q_search[0])
context = page.content

In [39]:
inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="tf")

Token indices sequence length is longer than the specified maximum sequence length for this model (7 > 512). Running this sequence through the model will result in indexing errors


In [40]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 30088), dtype=int32, numpy=array([[ 101, 2043, 2106, ..., 1997, 7649,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 30088), dtype=int32, numpy=array([[0, 0, 0, ..., 1, 1, 1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 30088), dtype=int32, numpy=array([[1, 1, 1, ..., 1, 1, 1]], dtype=int32)>}

In [41]:
# answering wikipedia questions

In [42]:
max_length = model.config.max_position_embeddings

In [43]:
max_length

512

In [44]:
question_mask = inputs["token_type_ids"] == 0

In [45]:
question_tensor = tf.boolean_mask(inputs["input_ids"], question_mask)

In [46]:
question_tensor

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 101, 2043, 2106, 2088, 2162, 1016, 2707, 1029,  102], dtype=int32)>

In [47]:
chunk_size = max_length - len(question_tensor) - 1

In [48]:
chunked_input = OrderedDict()

In [49]:
def get_chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [50]:
for k, v in inputs.items():
    question = tf.boolean_mask(v, question_mask)
    context =  tf.boolean_mask(v, ~question_mask)
    chunks = list(get_chunks(context, chunk_size))
    
    for i, chunk in enumerate(chunks):
        if i not in chunked_input:
            chunked_input[i] = {}
            
        entire_input = tf.concat([question, chunk], axis=0)
        
        if i != len(chunks) - 1:
            if k == 'input_ids':
                entire_input = tf.concat([entire_input, tf.constant([102])], axis=0)
            else:
                entire_input = tf.concat([entire_input, tf.constant([1])], axis=0)
        
        chunked_input[i][k] = tf.reshape(entire_input, [1, entire_input.shape[0]])

In [51]:
chunked_input[0]

{'input_ids': <tf.Tensor: shape=(1, 512), dtype=int32, numpy=
 array([[  101,  2043,  2106,  2088,  2162,  1016,  2707,  1029,   102,
          2088,  2162,  1045,  1006,  2411, 12066,  2004,  1059,  9148,
          2030,  1059,  2860,  2487,  1007,  1010,  2036,  2124,  2004,
          1996,  2034,  2088,  2162,  2030,  1996,  2307,  2162,  1010,
          2001,  1037,  3795,  2162,  2008,  6354,  2013,  2654,  2251,
          4554,  2000,  2340,  2281,  4271,  1012,  9530, 18532, 17822,
         17191,  2135,  2649,  2004,  1000,  1996,  2162,  2000,  2203,
          2035,  5233,  1000,  1010,  2009,  2419,  2000,  1996, 11240,
         24411,  3370,  1997,  2062,  2084,  3963,  2454,  2510,  5073,
          1010,  2164,  3438,  2454, 13481,  1010,  2437,  2009,  2028,
          1997,  1996,  2922,  5233,  1999,  2381,  1012,  2009,  2003,
          2036,  2028,  1997,  1996,  2757, 21292,  9755,  1999,  2381,
          1010,  2007,  2019,  4358,  3157,  2454,  4337,  4630,  6677,
  

In [52]:
answer = ''

for k, chunk in chunked_input.items():
    start_scores, end_scores = model(chunk)

    answer_start = tf.argmax(start_scores, axis=1).numpy()[0]
    answer_end = (tf.argmax(end_scores, axis=1) + 1).numpy()[0]
    answ = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(chunk['input_ids'][0][answer_start:answer_end]))
    
    if answ != '[CLS]':
        answer += answ + " / "

In [53]:
answer

'1939 / 21 march 1918 / '